In [1]:
import polars as pl

%load_ext autoreload
%autoreload 2

# Base Load Analysis Demo with Polars

This notebook demonstrates analyzing base load (standby power consumption) using the BaseloadAnalyzer class. Base load represents the minimum continuous power draw in a system.

## Key Metrics
1. Base load value in WATTS - Consistent minimum power draw
2. Energy consumption in kWh - Power used over time
3. Base load percentage - Portion of total consumption that is baseline

## Data Requirements
Input data (LazyFrame):
- timestamp: datetime with timezone
- total: energy readings in kWh (15-minute intervals)


> ⚠️**Note:** we have several example files available,  
> *energy_use_big* is from a giant building with incredible base load.  
> *energy_use_test1* is from a regular family residence.


In [2]:
from openenergyid.baseload.analysis import BaseloadAnalyzer

# Define schema for data loading
schema = {"timestamp": pl.Datetime(time_zone="Europe/Brussels"), "total": pl.Float64}

# Load example data with schema
energy_data = pl.scan_ndjson("data/PP/energy_use_test1.ndjson", schema=schema)

## Initialize Analyzer
Set up analyzer with timezone and quantile settings

In [3]:
# Create analyzer (5% quantile = ~72 min of lowest daily values)
analyzer = BaseloadAnalyzer(timezone="Europe/Brussels", quantile=0.15)

# Convert energy readings to power series
power_data = analyzer.prepare_power_seriespolars(energy_data)

## Analyze at Different Time Scales
Demonstrate flexibility in analysis periods

In [4]:
# Analyze at different granularities
hourly = analyzer.analyze(power_data, "1h").collect()
daily = analyzer.analyze(power_data, "1d").collect()
monthly = analyzer.analyze(power_data, "1mo").collect()

# Show monthly summary
print("Monthly Base Load Analysis:")
print(monthly.select(["timestamp", "average_daily_baseload_in_watt", "baseload_ratio"]).head())

Monthly Base Load Analysis:
shape: (5, 3)
┌───────────────────────────────┬────────────────────────────────┬────────────────┐
│ timestamp                     ┆ average_daily_baseload_in_watt ┆ baseload_ratio │
│ ---                           ┆ ---                            ┆ ---            │
│ datetime[μs, Europe/Brussels] ┆ f64                            ┆ f64            │
╞═══════════════════════════════╪════════════════════════════════╪════════════════╡
│ 2023-01-01 00:00:00 CET       ┆ 94.670428                      ┆ 0.20993        │
│ 2023-02-01 00:00:00 CET       ┆ 99.428571                      ┆ 0.215488       │
│ 2023-03-01 00:00:00 CET       ┆ 105.142857                     ┆ 0.223711       │
│ 2023-04-01 00:00:00 CEST      ┆ 99.333333                      ┆ 0.275984       │
│ 2023-05-01 00:00:00 CEST      ┆ 104.645161                     ┆ 0.259927       │
└───────────────────────────────┴────────────────────────────────┴────────────────┘


## Visualization Example
Plot daily base load vs total consumption

In [5]:
import plotly.express as px
import plotly.graph_objects as go

# Convert to pandas for plotting
daily_pd = daily.to_pandas()

# Create figure with basic lines
fig = px.line(
    daily_pd,
    x="timestamp",
    y=["consumption_due_to_baseload_in_kilowatthour", "total_consumption_in_kilowatthour"],
    title="Daily Base Load vs Total Consumption",
    labels={"value": "Energy (kWh)", "variable": "Type"},
)

# Add average lines
fig.add_hline(
    y=daily_pd["consumption_due_to_baseload_in_kilowatthour"].mean(),
    line_dash="dash",
    line_color="blue",
    annotation_text="Average Base Load",
)

fig.add_hline(
    y=daily_pd["total_consumption_in_kilowatthour"].mean(),
    line_dash="dash",
    line_color="red",
    annotation_text="Average Total Consumption",
)

# Update colors and legend
fig.update_traces(
    name="Base Load",
    line_color="blue",
    selector=dict(name="consumption_due_to_baseload_in_kilowatthour"),
)
fig.update_traces(
    name="Total Consumption",
    line_color="red",
    selector=dict(name="total_consumption_in_kilowatthour"),
)

fig.show()

In [6]:
import polars as pl
import plotly.express as px

# Load data with schema
schema = {"timestamp": pl.Datetime(time_zone="Europe/Brussels"), "total": pl.Float64}
energy_data = pl.scan_ndjson("data/PP/energy_use_test1.ndjson", schema=schema).sort("timestamp")


# Analyze with different quantiles
def analyze_quantile(q: float):
    analyzer = BaseloadAnalyzer(timezone="Europe/Brussels", quantile=q)
    power_data = analyzer.prepare_power_seriespolars(energy_data)
    return analyzer.analyze(power_data, "1d").collect()


# Get results for different quantiles
q05 = analyze_quantile(0.05)
q10 = analyze_quantile(0.10)
q15 = analyze_quantile(0.15)

# Create visualization
fig = go.Figure()

# Add lines for each quantile
for data, q in [(q05, "5%"), (q10, "10%"), (q15, "15%")]:
    fig.add_trace(
        go.Scatter(
            x=data["timestamp"],
            y=data["consumption_due_to_baseload_in_kilowatthour"],
            name=f"Base Load (q={q})",
            mode="lines",
        )
    )

# Add total consumption line
fig.add_trace(
    go.Scatter(
        x=q05["timestamp"],
        y=q05["total_consumption_in_kilowatthour"],
        name="Total Consumption",
        mode="lines",
        line=dict(color="gray", dash="dot"),  # Make it dotted gray to distinguish
    )
)

fig.update_layout(
    title="Base Load Comparison - Different Quantiles",
    xaxis_title="Date",
    yaxis_title="Energy (kWh)",
)

fig.show()

# Print average values
print("\nAverage Base Load (kWh):")
print(f"5% quantile: {q05['consumption_due_to_baseload_in_kilowatthour'].mean():.3f}")
print(f"10% quantile: {q10['consumption_due_to_baseload_in_kilowatthour'].mean():.3f}")
print(f"15% quantile: {q15['consumption_due_to_baseload_in_kilowatthour'].mean():.3f}")


Average Base Load (kWh):
5% quantile: 1.907
10% quantile: 2.266
15% quantile: 2.447


In [7]:
monthly
# daily

timestamp,consumption_due_to_baseload_in_kilowatthour,total_consumption_in_kilowatthour,average_daily_baseload_in_watt,average_power_in_watt,consumption_not_due_to_baseload_in_kilowatthour,baseload_ratio
"datetime[μs, Europe/Brussels]",f64,f64,f64,f64,f64,f64
2023-01-01 00:00:00 CET,70.340128,335.064964,94.670428,450.962266,264.724836,0.20993
2023-02-01 00:00:00 CET,66.816,310.068,99.428571,461.410714,243.252,0.215488
2023-03-01 00:00:00 CET,78.016,348.736,105.142857,469.994609,270.72,0.223711
2023-04-01 00:00:00 CEST,71.52,259.145,99.333333,359.923611,187.625,0.275984
2023-05-01 00:00:00 CEST,77.856,299.53,104.645161,402.594086,221.674,0.259927
…,…,…,…,…,…,…
2023-09-01 00:00:00 CEST,84.192,317.123,116.933333,440.448611,232.931,0.265487
2023-10-01 00:00:00 CEST,75.28,372.593,100.911528,499.454424,297.313,0.202044
2023-11-01 00:00:00 CET,70.56,342.402,98.0,475.558333,271.842,0.206074


## Key Insights
- Base load typically accounts for 20-40% of total consumption
- Higher ratios may indicate energy saving opportunities
- Analysis maintains timezone awareness throughout